In [1]:
from util import load_user_data
import pandas as pd
import numpy as np

# Load explanation understanding, no control condition
valid_users, tp_data, _ = load_user_data(filename="../data/xailabdata_all.csv")
valid_users_2, tp_data_2, _ = load_user_data(filename="../data/xailabdata_llm_agent.csv")

valid_users = valid_users | valid_users_2
tp_data.update(tp_data_2)

variable_dict = {}
variable_dict["condition"] = []
understanding_variable_dict = {}
understanding_variable_dict["condition"] = []
explanation_understanding_dimensions = ["perceived_feature_understanding", "completeness", "coherence", "clarity", \
    "learning_effect", "understanding_of_system", "usefulness_explanation", "objective_feature_understanding"]
covariates_dimensions = ["Propensity to Trust", "Familiarity", "ATI", "mlbackground"]
trust_dimensions = ["Reliability/Competence", "Understandability/Predictability", "Trust in Automation"]
reliance_dimensions = ["accuracy", "agreement_fraction", "switch_fraction", "accuracy-wid", "RAIR", "RSR"]
for dimension in reliance_dimensions:
    variable_dict[dimension] = []
for dimension in trust_dimensions:
    variable_dict[dimension] = []
for dimension in covariates_dimensions:
    variable_dict[dimension] = []
    understanding_variable_dict[dimension] = []
for dimension in explanation_understanding_dimensions:
    understanding_variable_dict[dimension] = []
for user in valid_users:
    tp_condition = tp_data[user]["condition"]
    variable_dict["condition"].append(tp_condition)

    # for users in all conditions, load reliance
    reliance_measures = tp_data[user]["reliance_measures"]
    for dimension in reliance_dimensions:
        variable_dict[dimension].append(reliance_measures[dimension])
    # for participants in all conditions, load covariates and trust
    trust = tp_data[user]["Trust_in_automation"]
    for dimension in trust_dimensions:
        variable_dict[dimension].append(trust[dimension])
    for dimension in ["Propensity to Trust", "Familiarity"]:
        variable_dict[dimension].append(trust[dimension])
    for dimension in ["ATI", "mlbackground"]:
        variable_dict[dimension].append(tp_data[user][dimension])
    if tp_condition == "control":
        continue
    # for participants in dashboard, chatxai and chatxaiboost conditions, load explanation udnerstanding
    explanation_understanding = tp_data[user]["explanation_understanding"]
    understanding_variable_dict["condition"].append(tp_condition)
    for dimension in explanation_understanding_dimensions:
        understanding_variable_dict[dimension].append(explanation_understanding[dimension])
    for dimension in ["Propensity to Trust", "Familiarity"]:
        understanding_variable_dict[dimension].append(trust[dimension])
    for dimension in ["ATI", "mlbackground"]:
        understanding_variable_dict[dimension].append(tp_data[user][dimension])

# for dimension in variable_dict:
#     print(dimension, len(variable_dict[dimension]))

df = pd.DataFrame(variable_dict)
df2 = pd.DataFrame(understanding_variable_dict)
print(df.shape)

245 valid participants
{'control': 61, 'dashboard': 61, 'chatxai': 62, 'chatxaiboost': 61, 'chatxaiAuto': 0}
91 participants blindly rely on AI advice
{'control': 8, 'dashboard': 27, 'chatxai': 32, 'chatxaiboost': 24, 'chatxaiAuto': 0}
61 valid participants
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 61}
25 participants blindly rely on AI advice
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 25}
(306, 14)


In [2]:
df2.head()

,condition,Propensity to Trust,Familiarity,ATI,mlbackground,perceived_feature_understanding,completeness,coherence,clarity,learning_effect,understanding_of_system,usefulness_explanation,objective_feature_understanding
0,chatxai,3.000000,1.0,3.666667,0,5,4.0,4,5,4,4,5.0,0.941976
1,chatxaiboost,3.000000,2.0,4.555556,1,4,3.5,5,5,5,4,3.9,0.993013
2,chatxai,3.333333,2.0,4.000000,0,3,4.0,4,4,4,3,3.9,0.758516
3,chatxaiboost,3.000000,2.0,3.000000,0,4,3.5,3,4,4,4,3.6,0.716540
4,chatxaiAuto,1.333333,2.0,5.111111,0,3,3.0,2,3,2,2,2.5,0.802893


In [3]:
from scipy.stats import spearmanr

# table rows: ["Propensity to Trust", "Familiarity", "ATI", "mlbackground"]
# corr, p; corr, p; corr, p; corr, p
# explanation understanding
# trust
# reliance

str_dict = {}
for dv in explanation_understanding_dimensions:
    str_dict[dv] = "{} ".format(dv)
    for cov in covariates_dimensions:
        correlation, pvalue = spearmanr(understanding_variable_dict[cov], understanding_variable_dict[dv])
        # if pvalue < 0.05 / 4:
        # print("Variable {} and variable {} have spearman correlation {:.3f} and pvalue {:.3f}".format(cov, dv, correlation, pvalue))
        # str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
        if pvalue < 0.0125:
            str_dict[dv] += "& {} &".format("%.3f"%correlation)
            str_dict[dv] += "\\textbf{"
            str_dict[dv] += "{}".format(("%.3f"%pvalue)[1:])
            str_dict[dv] += "}$^{\dagger\dagger}$"
        elif pvalue < 0.05:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
            str_dict[dv] += "$^{\dagger}$"
        else:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
    str_dict[dv] += "\\\\"
    print(str_dict[dv])
print("-" * 17)

for dv in trust_dimensions:
    str_dict[dv] = "{} ".format(dv)
    for cov in covariates_dimensions:
        correlation, pvalue = spearmanr(variable_dict[cov], variable_dict[dv])
        # if pvalue < 0.05 / 4:
        # print("Variable {} and variable {} have spearman correlation {:.3f} and pvalue {:.3f}".format(cov, dv, correlation, pvalue))
        # str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
        if pvalue < 0.0125:
            str_dict[dv] += "& {} &".format("%.3f"%correlation)
            str_dict[dv] += "\\textbf{"
            str_dict[dv] += "{}".format(("%.3f"%pvalue)[1:])
            str_dict[dv] += "}$^{\dagger\dagger}$"
        elif pvalue < 0.05:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
            str_dict[dv] += "$^{\dagger}$"
        else:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
    str_dict[dv] += "\\\\"
    print(str_dict[dv])
print("-" * 17)


for dv in reliance_dimensions:
    str_dict[dv] = "{} ".format(dv)
    for cov in covariates_dimensions:
        correlation, pvalue = spearmanr(variable_dict[cov], variable_dict[dv])
        # if pvalue < 0.05 / 4:
        # print("Variable {} and variable {} have spearman correlation {:.3f} and pvalue {:.3f}".format(cov, dv, correlation, pvalue))
        # str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
        if pvalue < 0.0125:
            str_dict[dv] += "& {} &".format("%.3f"%correlation)
            str_dict[dv] += "\\textbf{"
            str_dict[dv] += "{}".format(("%.3f"%pvalue)[1:])
            str_dict[dv] += "}$^{\dagger\dagger}$"
        elif pvalue < 0.05:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
            str_dict[dv] += "$^{\dagger}$"
        else:
            str_dict[dv] += "& {} & {}".format("%.3f"%correlation, ("%.3f"%pvalue)[1:])
    str_dict[dv] += "\\\\"
    print(str_dict[dv])
print("-" * 17)


perceived_feature_understanding & 0.344 &\textbf{.000}$^{\dagger\dagger}$& 0.131 & .041$^{\dagger}$& 0.148 & .021$^{\dagger}$& 0.049 & .444\\
completeness & 0.366 &\textbf{.000}$^{\dagger\dagger}$& 0.106 & .097& 0.073 & .254& 0.152 & .017$^{\dagger}$\\
coherence & 0.387 &\textbf{.000}$^{\dagger\dagger}$& 0.131 & .040$^{\dagger}$& 0.087 & .175& 0.135 & .035$^{\dagger}$\\
clarity & 0.427 &\textbf{.000}$^{\dagger\dagger}$& 0.069 & .285& 0.129 & .044$^{\dagger}$& 0.142 & .026$^{\dagger}$\\
learning_effect & 0.232 &\textbf{.000}$^{\dagger\dagger}$& 0.173 &\textbf{.007}$^{\dagger\dagger}$& 0.115 & .072& 0.147 & .021$^{\dagger}$\\
understanding_of_system & 0.343 &\textbf{.000}$^{\dagger\dagger}$& 0.082 & .202& 0.146 & .022$^{\dagger}$& 0.080 & .210\\
usefulness_explanation & 0.423 &\textbf{.000}$^{\dagger\dagger}$& 0.166 &\textbf{.009}$^{\dagger\dagger}$& 0.172 &\textbf{.007}$^{\dagger\dagger}$& 0.083 & .196\\
objective_feature_understanding & 0.108 & .092& -0.152 & .017$^{\dagger}$& 0.013 & 